Import the necessary libraries

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
# import skimage as sky
import matplotlib.pyplot as plt
import os
import random
import shutil
from scipy import ndimage
from joblib import Parallel, delayed
import tensorflow as tf
from sklearn.model_selection import KFold

In [2]:
tf.config.list_physical_devices ('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Create the folds based on the stacked np arrays

In [3]:
# load the numpy files of the images and the sub image counts

# data files
all_train_sub_images = np.load(r"/home/statsgeneral/gayara/Tasselnet/Filtered_90/Data_files/all_sub_windows_float16.npy")

In [4]:
all_train_sub_images.shape

(1347113, 32, 32, 3)

In [5]:
all_train_sub_images_counts = np.load(r"/home/statsgeneral/gayara/Tasselnet/Filtered_90/Data_files/all_sub_windows_counts_float16.npy")

In [6]:
all_train_sub_images_counts.shape

(1347113,)

In [24]:
all_train_sub_images_counts = all_train_sub_images_counts.astype('float16')
all_train_sub_images = all_train_sub_images.astype('float16')

MemoryError: Unable to allocate 7.71 GiB for an array with shape (1347113, 32, 32, 3) and data type float16

In [7]:
# generate a random sample based on a sequence

# fix the seed
random.seed(32)

# all data
rand_all_available = random.sample(range(len(all_train_sub_images)),len(all_train_sub_images))


In [8]:
# extract the 269423 data points and the corresponding counts
train_np_arrays= [all_train_sub_images[index] for index in rand_all_available]
train_sub_counts = [all_train_sub_images_counts[index] for index in rand_all_available]

In [9]:
# stack the files all together
data_final_np = np.stack(train_np_arrays)

In [10]:
data_final_np.shape

(1347113, 32, 32, 3)

In [11]:
counts_final_np = np.stack(train_sub_counts)

In [12]:
counts_final_np.shape

(1347113,)

In [13]:
kf = KFold(n_splits=5)

In [14]:
X_train_all = []
X_test_all = []
y_train_all = []
y_test_all = []
for train_index, test_index in kf.split(data_final_np):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = data_final_np[train_index], data_final_np[test_index]
    X_train_all.append(X_train)
    X_test_all.append(X_test)
    y_train, y_test = counts_final_np[train_index], counts_final_np[test_index]
    y_train_all.append(y_train)
    y_test_all.append(y_test)

TRAIN: [ 269423  269424  269425 ... 1347110 1347111 1347112] TEST: [     0      1      2 ... 269420 269421 269422]
TRAIN: [      0       1       2 ... 1347110 1347111 1347112] TEST: [269423 269424 269425 ... 538843 538844 538845]
TRAIN: [      0       1       2 ... 1347110 1347111 1347112] TEST: [538846 538847 538848 ... 808266 808267 808268]
TRAIN: [      0       1       2 ... 1347110 1347111 1347112] TEST: [ 808269  808270  808271 ... 1077688 1077689 1077690]
TRAIN: [      0       1       2 ... 1077688 1077689 1077690] TEST: [1077691 1077692 1077693 ... 1347110 1347111 1347112]


In [15]:
X_train_all[4].shape

(1077691, 32, 32, 3)

In [18]:
# Store_path = r"/work/statsgeneral/gayara/Tassel_datasets_F90"

In [19]:
# counter = 1
# for data_set in X_train_all:
#     data = data_set
#     name = 'x_train_data_' + str(counter) 
#     np.save(Store_path + "/" + name, data)
#     counter = counter + 1
    

In [21]:
# counter = 1
# for data_set in X_test_all:
#     data_n = data_set
#     name = 'x_test_data_' + str(counter) 
#     np.save(Store_path + "/" + name, data_n)
#     counter = counter + 1

In [22]:
# counter = 1
# for data_set in y_train_all:
#     data_n = data_set
#     name = 'y_train_' + str(counter) 
#     np.save(Store_path + "/" + name, data_n)
#     counter = counter + 1

In [23]:
# counter = 1
# for data_set in y_test_all:
#     data_n = data_set
#     name = 'y_test_' + str(counter) 
#     np.save(Store_path + "/" + name, data_n)
#     counter = counter + 1

Do model training

In [18]:
# need to introduce a counter for saving the models
counter = 1
for i in range(len(X_train_all)):
    X_train = X_train_all[i]
    y_train = y_train_all[i]
    X_test = X_test_all[i]
    y_test = y_test_all[i]
    # load the pre-trained model on tassels
    model = tf.keras.models.load_model('/home/statsgeneral/gayara/Tasselnet/Filtered_90/tasselnet1_overlapping_w32.h5')
    # look at the input shape
    model.input
    
    # need to remove some of the last layers
    output_trial = model.layers[-8].output
    
# create the functional API model
    reduced_model = tf.keras.models.Model(model.input, output_trial)

# okay, now need to add back the dropout, the dense and activation

# add dropout
    added_dropout = tf.keras.layers.Dropout(0.5, name = "New_dropout")(model.layers[-8].output)

# add flatten
    added_flatten = tf.keras.layers.Flatten(name = "Flatten2")(added_dropout)

# add dense
    added_dense = tf.keras.layers.Dense(512, name = "New_Dense")(added_flatten)

# add activation
    added_Act = tf.keras.layers.Activation('relu', name = "New_Activation")(added_dense)

# add dropout
    added_dropout2 = tf.keras.layers.Dropout(0.5, name = "New_dropout2")(added_Act)

# add dense
    added_dense2 = tf.keras.layers.Dense(1, name = "New_Dense2")(added_dropout2)

# add activation
    added_Act2 = tf.keras.layers.Activation('relu', name = "New_Activation2")(added_dense2)
    
# define the new model with functional API
    new_model = tf.keras.models.Model(model.input, added_Act2)
    
#     freeze layers
    for layer in model.layers:
        layer.trainable = False
    
# compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
# add early stopping
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=5)
    
#     fit the frozen model:
    history = new_model.fit(X_train, y_train,
          epochs = 50, callbacks = [es],
          validation_data = (X_test, y_test), 
                       batch_size = 64, validation_batch_size = 64)
    
    
# # unfreeze few layers and retrain
    model.trainable = True

    set_trainable = False

    for layer in model.layers:
        if layer.name == 'conv2d_3':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
            
# # compile the mdoel
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
# # add early stopping
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=5)
    
# # fit the model (fine tuned)
    history_new = new_model.fit(X_train, y_train,
          epochs = 50, callbacks = [es],
          validation_data = (X_test, y_test), 
                       batch_size = 2000, validation_batch_size = 2000)
    
    #     save the model
    model_name = 'model_' + str(counter) + '.h5'
    counter = counter + 1
    new_model.save('/work/statsgeneral/gayara/tasselnet/models' + '/' + model_name)

MemoryError: Unable to allocate 12.3 GiB for an array with shape (1077690, 32, 32, 3) and data type float32